In [7]:
import numpy as np

In [9]:
def read_input(name = 'input.txt'): 
    with open(name, "r") as f:
        n = int(f.readline().strip())
        a = np.zeros((n, n), dtype=float)
        b = np.zeros(n, dtype=float)
        for i in range(n):
            a[i] = np.fromstring(f.readline(), sep=" ")
        b = np.fromstring(f.readline(), sep=" ")
    
    return a, b

In [11]:
def write_output(x, true_x, name = "output.txt"):
    abs_err = np.sqrt(np.sum((x[0] - true_x) ** 2))
    rel_err = abs_err / np.sqrt(np.sum(true_x ** 2))

    with open(name, "w") as f:
        f.write("My Result:\t" + np.array2string(x[0], precision=9, separator=', ') + f"\tDeterminant: {x[1][0]} * e^{x[1][1]}\n")
        f.write("NumPy Result: " + np.array2string(true_x, precision=9, separator=', ') + "\n")
        f.write(f"Absolute error: {abs_err:.6e}\n")
        f.write(f"Relative error: {rel_err:.6e}\n")

## Додаткове завдання

In [13]:
def find_pivot(a, k):
    matrix = np.abs(a[k:, k:])
    max_idx_flat = matrix.argmax()
    max_idx = np.unravel_index(max_idx_flat, matrix.shape)
    return max_idx[0]+k, max_idx[1]+k

In [15]:
def gauss(A, B):
    
    #Creating variables
    a = A.astype(float).copy()
    b = B.astype(float).copy()
    n = len(a)
    logdet = np.longdouble(0.)
    sign = 1
    col_order = list(range(n))
    x = np.zeros(n, dtype=float)
    
    # Main pivoting algorythm
    for k in range(n):
        row, col = find_pivot(a, k)     #look for max value in matrix
        if row != k:
            sign *= -1
            a[[k, row]] = a[[row, k]]       # Swap rows
            b[k], b[row] = b[row], b[k]     # Swap rows
        if col != k:
            sign *= -1
            a[:, [k, col]] = a[:, [col, k]]                             # Swap cols
            col_order[k], col_order[col] = col_order[col], col_order[k] # Remember order
            
        if a[k][k] == 0: raise np.linalg.LinAlgError("Матриця вироджена")  

        #A loop for making zeros
        for i in range(k+1, n):
            koef = a[i][k] / a[k][k]
            b[i] -= koef*b[k]
            a[i][k:n] = a[i][k:n] - koef * a[k][k:n]      
            
        # Determinant calculating
        if a[k][k] < 0:
            sign *= -1
            logdet += np.log(abs(a[k][k]))
        else:
            logdet += np.log(a[k][k])
    
    # Substitution
    for i in range(n-1, -1, -1):
        if a[i][i] == 0: raise np.linalg.LinAlgError("Матриця вироджена")
        x[i] = (b[i] - np.dot(a[i][i+1:n], x[i+1:n]))/a[i][i]
        
    # Reordering the result
    result = np.empty_like(x)
    for i in range(n):
        result[col_order[i]] = x[i]
    return result, (sign , logdet)
        

In [1]:
def test_gauss(num_tests=5, n=1000, seed=357902750):
    np.random.seed(seed)
    for t in range(num_tests):
        A = np.random.randn(n, n) 
        x_true = np.random.randn(n)
        b = np.dot(A, x_true)

        try:
            x_gauss = gauss(A, b)[0]
            x_numpy = np.linalg.solve(A, b)
        except np.linalg.LinAlgError as e:
            print(f"Test {t+1}: Singular matrix detected ({e})")
            continue

        ok = np.allclose(x_gauss, x_numpy, atol=1e-9)
        err = np.linalg.norm(x_gauss - x_numpy)
        print(f"Test {t+1}: {'PASS' if ok else 'FAIL'} | Error norm = {err:.2e}")


In [30]:
test_gauss()

Test 1: PASS | Error norm = 1.73e-10
Test 2: PASS | Error norm = 2.48e-10
Test 3: PASS | Error norm = 5.06e-12
Test 4: PASS | Error norm = 4.48e-12
Test 5: PASS | Error norm = 1.68e-11
